[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/recommendation/product-recommender/product_recommender.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/recommendation/product-recommender/product_recommender.ipynb)

# Product Recommendation Engine

Learn how to build a product recommendation engine using collaborative filtering and Pinecone.

In this example, we will generate product recommendations for ecommerce customers based on previous orders and trending items. This example covers preparing the vector embeddings, creating and deploying the Pinecone service, writing data to Pinecone, and finally querying Pinecone to receive a ranked list of recommended products.

---

🚨 _Note that running this on CPU is slow! If running on Google Colab you go to **Runtime > Change runtime type > Hardware accelerator > GPU** to switch to GPU._

---

## Data Preparation

**Import Python Libraries**

In [1]:
import os
import time
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import itertools

In [ ]:
!pip install -qU kaggle

In [ ]:
try:
    import kaggle
except OSError as e:
    print(e)

The first time you `import kaggle` you will see an `OSError`, that is because we need to add our Kaggle credentials to the `/root/.kaggle/kaggle.json` file. You can find these credentials on [Kaggle](https://kaggle.com) by accessing your profile in the top-right corner of the page. This will download a `kaggle.json` file which contains your username and secret key. You can enter them below:

In [ ]:
import json

with open('/root/.kaggle/kaggle.json', 'w') as fp:
    fp.write(json.dumps({"username":"YOUR_USERNAME","key":"YOUR_SECRET_KEY"}))

Now we can download the dataset:

In [ ]:
!kaggle competitions download -c instacart-market-basket-analysis

This downloads a set of zip files, we extract them like so:

In [ ]:
import zipfile

files = [
    'instacart-market-basket-analysis.zip',
    'order_products__train.csv.zip',
    'order_products__prior.csv.zip',
    'products.csv.zip',
    'orders.csv.zip'
]

for filename in files:
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('./')

Now we can move on to loading the dataset.

**Load the (Example) Instacart Data**

We are going to use the [Instacart Market Basket Analysis](https://www.kaggle.com/c/instacart-market-basket-analysis/data) dataset for this task.

The data used throughout this example is a set of files describing customers' orders over time. The main focus is on the *orders.csv* file, where each line represents a relation between a user and the order. In other words, each line has information on *userid* (user who made the order) and *orderid*. Note there is no information about products in this table. Product information related to specific orders is stored in the *order_product__*.csv* dataset.

In [2]:
order_products_train = pd.read_csv('order_products__train.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')

order_products = pd.concat([order_products_train, order_products_prior])

**Preparing data for the model**


The Collaborative Filtering model used in this example requires only users’ historical preferences on a set of items. As there is no explicit rating in the data we are using, the purchase quantity can represent a “confidence” in terms of how strong the interaction was between the user and the products.

The dataframe data will store this data and will be the base for the model.

In [3]:
customer_order_products = pd.merge(orders, order_products, how='inner',on='order_id')

# creating a table with "confidences"
data = customer_order_products.groupby(['user_id', 'product_id'])[['order_id']].count().reset_index()
data.columns=["user_id", "product_id", "total_orders"]
data.product_id = data.product_id.astype('int64')

# Create a lookup frame so we can get the product names back in readable form later.
products_lookup = products[['product_id', 'product_name']].drop_duplicates()
products_lookup['product_id'] = products_lookup.product_id.astype('int64')

We will create two prototype users here and add them to our data dataframe. Each user will be buying only a specific product:
- The first user will be buying only **Mineral Water**
- The second user will be buying baby products: **No More Tears Baby Shampoo** and **Baby Wash & Shampoo**

These users will be later used for querying and examination of the model results.

In [4]:
data_new = pd.DataFrame([[data.user_id.max() + 1, 22802, 97],
                         [data.user_id.max() + 2, 26834, 89],
                         [data.user_id.max() + 2, 12590, 77]
                        ], columns=['user_id', 'product_id', 'total_orders'])
data_new

,user_id,product_id,total_orders
0,206210,22802,97
1,206211,26834,89
2,206211,12590,77


In [5]:
data = pd.concat([data, data_new]).reset_index(drop = True)
data.tail()

,user_id,product_id,total_orders
13863744,206209,48697,1
13863745,206209,48742,2
13863746,206210,22802,97
13863747,206211,26834,89
13863748,206211,12590,77


In the next step, we will first extract user and item unique ids, in order to create a CSR (Compressed Sparse Row) matrix. 


In [6]:
users = list(np.sort(data.user_id.unique()))
items = list(np.sort(products.product_id.unique()))
purchases = list(data.total_orders)

# create zero-based index position <-> user/item ID mappings
index_to_user = pd.Series(users)

# create reverse mappings from user/item ID to index positions
user_to_index = pd.Series(data=index_to_user.index + 1, index=index_to_user.values)

# create zero-based index position <-> item/user ID mappings
index_to_item = pd.Series(items)

# create reverse mapping from item/user ID to index positions
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Get the rows and columns for our new matrix
products_rows = data.product_id.astype(int)
users_cols = data.user_id.astype(int)

# Create a sparse matrix for our users and products containing number of purchases
sparse_product_user = sparse.csr_matrix((purchases, (products_rows, users_cols)), shape=(len(items) + 1, len(users) + 1))
sparse_product_user.data = np.nan_to_num(sparse_product_user.data, copy=False)

sparse_user_product = sparse.csr_matrix((purchases, (users_cols, products_rows)), shape=(len(users) + 1, len(items) + 1))
sparse_user_product.data = np.nan_to_num(sparse_user_product.data, copy=False)

## Implicit Model

In this section we will demonstrate creation and training of a recommender model using the **implicit** library. The recommendation model is based off the algorithms described in the paper [Collaborative Filtering for Implicit Feedback Datasets](https://www.researchgate.net/publication/220765111_Collaborative_Filtering_for_Implicit_Feedback_Datasets) with performance optimizations described in [Applications of the Conjugate Gradient Method for Implicit Feedback Collaborative Filtering](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.379.6473&rep=rep1&type=pdf).


In [7]:
!pip install --qU implicit

You should consider upgrading via the '/home/jelena/.pyenv/versions/3.8.16/envs/pinecone-onboarding/bin/python3.8 -m pip install --upgrade pip' command.


In [8]:
import implicit
from implicit import evaluation

#split data into train and test sets
train_set, test_set = evaluation.train_test_split(sparse_user_product, train_percentage=0.9)

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=100,
                                             regularization=0.05,
                                             iterations=50,
                                             num_threads=1)

alpha_val = 15
train_set = (train_set * alpha_val).astype('double')

# train the model on a sparse matrix of item/user/confidence weights
model.fit(train_set, show_progress = True)

  0%|          | 0/50 [00:00<?, ?it/s]

We will evaluate the model using the inbuilt library function

In [9]:
test_set = (test_set * alpha_val).astype('double')
evaluation.ranking_metrics_at_k(model, train_set, test_set, K=100,
                         show_progress=True, num_threads=1)

  0%|          | 0/192950 [00:00<?, ?it/s]

{'precision': 0.2744222469217778,
 'map': 0.043907108017036144,
 'ndcg': 0.14338860013526844,
 'auc': 0.6545833891054691}

This is what item and user factors look like. These vectors will be stored in our vector index later and used for recommendation.

In [10]:
model.item_factors[1:3]

array([[ 2.0967076e-02,  1.5675681e-02,  2.7659893e-02, -9.3854545e-03,
         3.9482825e-03, -3.6112599e-03,  1.9659773e-02,  9.5190024e-03,
        -8.8245980e-03,  9.2591681e-03,  2.4385843e-02, -1.5799325e-02,
        -3.5724377e-03,  1.3623893e-02,  2.5174504e-03, -4.2102817e-03,
         1.7773550e-02,  8.3679482e-03,  7.1635228e-03,  9.5603624e-03,
         2.1269219e-03,  2.5458889e-02, -5.4615783e-03, -1.3147385e-02,
         6.6509885e-03,  3.9727605e-05,  1.9329023e-02,  2.4163781e-02,
         5.7352367e-03,  6.9719194e-03,  1.1364637e-02,  1.5393358e-02,
         2.2142604e-02,  2.2375317e-02,  1.8471925e-02,  1.3722002e-02,
        -1.4020128e-02,  1.7060051e-02,  8.3400952e-03, -1.1008831e-03,
         8.4838523e-03,  1.4199644e-02,  8.0545079e-03,  1.0264770e-03,
         1.0547349e-02,  7.4009495e-03, -1.0270549e-02, -1.2610539e-02,
         1.7277125e-02,  1.3249416e-02,  2.4677438e-03,  5.2130860e-03,
        -2.1214210e-03,  4.3514427e-03, -1.8082142e-02,  6.93934

In [11]:
model.user_factors[1:3]

array([[-0.4243864 ,  1.275311  ,  2.2945623 , -1.9221066 , -0.05180068,
        -0.8445938 ,  1.0855463 ,  0.01313354, -1.0101424 , -0.20893407,
         0.2812347 , -1.3227096 ,  0.4357777 ,  0.4142942 , -1.331063  ,
        -0.06135667, -0.13825576, -0.13200262, -1.1354909 , -0.8584241 ,
        -1.3404921 , -0.2551998 ,  0.48202154,  0.2657751 ,  0.69813156,
        -0.79010475, -0.4260205 ,  2.703489  , -0.5978345 ,  1.6247606 ,
        -1.0061824 ,  0.32616058,  1.1687564 ,  1.4741871 ,  1.5606699 ,
         0.2605033 , -1.6973283 ,  1.1689923 , -0.1315944 , -1.1848983 ,
        -0.0037278 , -0.6783408 ,  0.98316395, -1.9251509 ,  0.5142688 ,
         1.0356346 ,  0.39273322, -1.5952505 , -0.2666866 ,  0.3976487 ,
         0.7356313 ,  0.37357202,  1.4322765 , -0.649824  , -0.8761418 ,
        -0.21579793,  1.1726382 ,  0.33400884,  1.0005614 , -0.60032487,
         2.6428957 , -0.26784444, -0.7223507 ,  1.2930341 , -0.42270628,
        -0.5213422 ,  0.6155089 ,  0.76245135,  1.2

## Configure Pinecone

Install and setup Pinecone

In [12]:
!pip install --qU pinecone-client

You should consider upgrading via the '/home/jelena/.pyenv/versions/3.8.16/envs/pinecone-onboarding/bin/python3.8 -m pip install --upgrade pip' command.


In [13]:
import pinecone

In [14]:
# Load Pinecone API key
api_key = os.getenv('PINECONE_API_KEY') or 'YOUR_API_KEY'
# Set Pinecone environment. Find next to API key in console
env = os.getenv('PINECONE_ENVIRONMENT') or "YOUR_ENV"

pinecone.init(api_key=api_key, environment=env)

[Get a Pinecone API key](http://app.pinecone.io/) if you don't have one.

In [15]:
#List all present indexes associated with your key, should be empty on the first run
pinecone.list_indexes()

[]

**Create an Index**

In [16]:
# Set a name for your index
index_name = 'product-recommender'

In [17]:
# Make sure service with the same name does not exist
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
pinecone.create_index(name=index_name, dimension=100)

**Connect to the new index**

In [18]:
index = pinecone.Index(index_name=index_name)

## Load Data

Uploading all items (products that one can buy) and displaying some examples of products and their vector representations.


In [19]:
# Get all of the items
all_items_titles = [{'title': title} for title in products_lookup['product_name']]
all_items_ids = [str(product_id) for product_id in products_lookup['product_id']]

# Transform items into factors
items_factors = model.item_factors

# Prepare item factors for upload
items_to_insert = list(zip(all_items_ids, items_factors[1:].to_numpy().tolist(), all_items_titles))
display(items_to_insert[:2])

[('1',
  [0.02096707560122013,
   0.015675680711865425,
   0.027659893035888672,
   -0.009385454468429089,
   0.003948282450437546,
   -0.003611259860917926,
   0.019659772515296936,
   0.00951900240033865,
   -0.008824598044157028,
   0.009259168058633804,
   0.02438584342598915,
   -0.015799324959516525,
   -0.0035724376793950796,
   0.01362389326095581,
   0.0025174503680318594,
   -0.004210281651467085,
   0.017773550003767014,
   0.008367948234081268,
   0.007163522765040398,
   0.009560362435877323,
   0.0021269218996167183,
   0.025458889082074165,
   -0.005461578257381916,
   -0.013147384859621525,
   0.006650988478213549,
   3.9727605326334015e-05,
   0.019329022616147995,
   0.024163780733942986,
   0.005735236685723066,
   0.006971919443458319,
   0.011364636942744255,
   0.01539335772395134,
   0.022142603993415833,
   0.0223753172904253,
   0.01847192458808422,
   0.013722001574933529,
   -0.014020128175616264,
   0.01706005074083805,
   0.008340095169842243,
   -0.0011008

**Insert items into the index**

In [23]:
from tqdm.auto import tqdm

BATCH_SIZE = 100

print('Index statistics before upsert:', index.describe_index_stats())

for i in tqdm(range(0, len(items_to_insert), BATCH_SIZE)):
        index.upsert(vectors=items_to_insert[i:i+BATCH_SIZE])

print('Index statistics after upsert:', index.describe_index_stats())

Index statistics before upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


  0%|          | 0/497 [00:00<?, ?it/s]

Index statistics after upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 49688}},
 'total_vector_count': 49688}


This is a helper method for analysing recommendations later.
This method returns top N products that someone bought in the past (based on product quantity).

In [24]:
def products_bought_by_user_in_the_past(user_id: int, top: int = 10):

    selected = data[data.user_id == user_id].sort_values(by=['total_orders'], ascending=False)

    selected['product_name'] = selected['product_id'].map(products_lookup.set_index('product_id')['product_name'])
    selected = selected[['product_id', 'product_name', 'total_orders']].reset_index(drop=True)
    if selected.shape[0] < top:
        return selected

    return selected[:top]

In [25]:
data.tail()

,user_id,product_id,total_orders
13863744,206209,48697,1
13863745,206209,48742,2
13863746,206210,22802,97
13863747,206211,26834,89
13863748,206211,12590,77


## Query for Recommendations

We are now retrieving user factors for users that we have manually created before for testing purposes. Besides these users, we are adding a random existing user. We are also displaying these users so you can see what these factors look like.

In [26]:
user_ids = [206210, 206211, 103593]
user_factors = model.user_factors[user_to_index[user_ids]]

display(user_factors[1:])

array([[ 2.1058037 ,  0.01496392,  0.6472299 , -0.12749648,  1.7000914 ,
         0.63003033, -0.44502732,  2.3930714 ,  1.5038052 , -1.4855131 ,
         0.1813116 , -0.27024838, -2.1634114 , -0.7732514 , -0.7400107 ,
         0.09586903,  0.24753384, -0.1307145 ,  2.1177993 , -1.1922722 ,
        -1.9040838 , -1.8427142 ,  3.3897648 , -0.09261613,  0.7281503 ,
        -0.25183126, -1.2083504 , -1.1893312 , -1.6289816 , -0.5372712 ,
        -0.5155484 ,  1.9106213 , -0.16083527, -0.65575653,  0.9658018 ,
        -1.686444  , -0.23160203,  2.3291728 ,  1.2048134 , -0.08264191,
        -0.829397  , -0.94631314, -2.0893824 ,  0.58197784,  2.0402133 ,
         1.2590383 , -2.245782  ,  1.2001114 ,  1.9336653 , -0.60793036,
         1.7353134 , -0.64028996,  1.6792128 , -1.848569  , -1.0521588 ,
         1.0003314 , -0.12361024, -0.22226357,  0.56298137,  0.4318341 ,
        -2.9131813 ,  0.08957183,  1.9043449 ,  0.07440718,  0.05985835,
        -2.047578  , -0.0182818 ,  2.198715  , -0.2

### Model recommendations

We will now retrieve recommendations from our model directly, just to have these results as a baseline.

In [27]:
print("Model recommendations\n")

start_time = time.process_time()
recommendations0 = model.recommend(userid=user_ids[0], user_items=sparse_user_product[0])
recommendations1 = model.recommend(userid=user_ids[1], user_items=sparse_user_product[1])
recommendations2 = model.recommend(userid=user_ids[2], user_items=sparse_user_product[2])
print("Time needed for retrieving recommended products: " + str(time.process_time() - start_time) + ' seconds.\n')

print('\nRecommendations for person 0:')
for recommendation in recommendations0[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

print('\nRecommendations for person 1:')
for recommendation in recommendations1[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

print('\nRecommendations for person 2:')
for recommendation in recommendations2[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

Model recommendations

Time needed for retrieving recommended products: 0.1621812180001143 seconds.


Recommendations for person 0:
['Mineral Water']
['Sparkling Water']
['Soda']
['Zero Calorie Cola']
['Smartwater']
['Sparkling Mineral Water']
['Drinking Water']
['Coconut Water']
['Organic Coconut Water']
['Spring Water']

Recommendations for person 1:
['Baby Wipes Sensitive']
['YoKids Squeezers Organic Low-Fat Yogurt, Strawberry']
['Organic Garbanzo Beans']
['Organic Cheese Frozen Pizza']
['Free and Gentle High Efficiency Liquid Laundry Detergent']
['Eggo Pancakes Minis']
['Strawberry Explosion/Banana Split Smoothie']
['Baby Wash & Shampoo']
['Danimals Strawberry Explosion Flavored Smoothie']
['No More Tears Baby Shampoo']

Recommendations for person 2:
['Organic Blackberries']
['Fat Free Milk']
['Clementines, Bag']
['Organic Cripps Pink Apples']
['Organic Bosc Pear']
['Red Plums']
['Organic Strawberries']
['Peach']
['Citrus Mandarins Organic']
['Blood Oranges']


### Query the index

Let's now query the index to check how quickly we retrieve results. Please note that query speed depends in part on your internet connection.

In [28]:
# Query by user factors

start_time = time.process_time()
query_results = index.query(queries=user_factors.to_numpy()[:-1].tolist(), top_k=10, include_metadata=True)
print("Time needed for retrieving recommended products using Pinecone: " + str(time.process_time() - start_time) + ' seconds.\n')

for _id, res in zip(user_ids, query_results.results):
    print(f'user_id={_id}')
    df = pd.DataFrame(
        {
            'products': [match.metadata['title'] for match in res.matches],
            'scores': [match.score for match in res.matches]
        }
    )
    print("Recommendation: ")
    display(df)
    print("Top buys from the past: ")
    display(products_bought_by_user_in_the_past(_id, top=15))

Time needed for retrieving recommended products using Pinecone: 0.02134626999941247 seconds.

user_id=206210
Recommendation: 


,products,scores
0,Mineral Water,0.917760
1,Zero Calorie Cola,0.707625
2,Popcorn,0.630404
3,Organic Coconut Water,0.619142
4,Drinking Water,0.610339
5,Orange & Lemon Flavor Variety Pack Sparkling F...,0.604182
6,Extra Fancy Unsalted Mixed Nuts,0.601729
7,Trail Mix,0.593162
8,Milk Chocolate Almonds,0.587405
9,Sparkling Water,0.582343


Top buys from the past: 


,product_id,product_name,total_orders
0,22802,Mineral Water,97


user_id=206211
Recommendation: 


,products,scores
0,Baby Wash & Shampoo,0.706027
1,No More Tears Baby Shampoo,0.682153
2,Size 6 Baby Dry Diapers,0.579626
3,Baby Wipes Sensitive,0.530385
4,Head-to-Toe Baby Wash,0.510900
5,Stage 1 Newborn Hypoallergenic Liquid Detergent,0.508379
6,Rockin' Raspberry 3.1 Fl Oz,0.502775
7,1pk 270ct Refill,0.496146
8,Size 5 Cruisers Diapers Super Pack,0.486226
9,White Buttermints,0.475889


Top buys from the past: 


,product_id,product_name,total_orders
0,26834,No More Tears Baby Shampoo,89
1,12590,Baby Wash & Shampoo,77


*Note* The inference using Pinecone is much faster compared to retrieving recommendations from a model directly. Please note that this result depends on your internet connection as well. 

All that’s left to do is surface these recommendations on the shopping site, or feed them into other applications.

## Clean up

Delete the index once you are sure that you do not want to use it anymore. Once it is deleted, you cannot reuse it.

In [29]:
pinecone.delete_index(index_name)

## Summary

In this example we used [Pinecone](https://www.pinecone.io/) to build and deploy a product recommendation engine that uses collaborative filtering, relatively quickly.

Once deployed, the product recommendation engine can index new data, retrieve recommendations in milliseconds, and send results to production applications.